# Importing packages

In [ ]:
import torch

from tokenizers import Tokenizer

from nmt.utils import config as config_util
from nmt.translator import Translator
from transformer import build_transformer

# Loading config file

In [ ]:
config_path = './config/inference.yaml'
config = config_util.get_config(config_path)

# Building `Translator` 

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

print('Loading tokenizers')
src_tokenizer = Tokenizer.from_file(config['src_tokenizer'])
target_tokenizer = Tokenizer.from_file(config['target_tokenizer'])

print('Loading model')
checkpoint_states = torch.load(config['checkpoint'], map_location=device)
required_keys = [
    'model_state_dict',
    'config',
]
for key in required_keys:
    if key not in checkpoint_states:
        raise ValueError(f'Missing key "{key}" in checkpoint')

transformer_config = checkpoint_states['config']
transformer_config.device = device
model = build_transformer(transformer_config).to(device)
model.load_state_dict(checkpoint_states['model_state_dict'])

translator = Translator(model, src_tokenizer, target_tokenizer, config)

Loading tokenizers
Loading model


In [4]:
translator("hello, world!", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=config['seq_length'])

['xin chào, thế giới!', 'xin chào!', 'xin chào thế giới!']

In [5]:
translator("Does he know about this?", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=config['seq_length'])

['anh ấy có biết không?', 'anh ta có biết không?', 'ông ấy có biết không?']

In [6]:
translator("i don't know how to do that.", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=config['seq_length'])

['tôi không biết làm thế nào.',
 'tôi không biết làm thế nào để làm điều đó.',
 'tôi không biết làm thế nào để làm được điều đó.']

In [7]:
translator("i don't know how to do these.", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=config['seq_length'])

['tôi không biết làm thế nào để làm việc này.',
 'tôi không biết làm thế nào để làm được điều này.',
 'tôi không biết làm thế nào để làm những điều này.']

In [8]:
translator("it's broken, great!", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=config['seq_length'])

['nó đã hỏng, tuyệt vời!', 'nó bị hỏng, tuyệt vời!', 'nó đã vỡ, tuyệt vời!']

In [9]:
translator("You would be surprised how kids are affected by this", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=config['seq_length'])

['bạn sẽ ngạc nhiên khi trẻ em bị ảnh hưởng bởi điều này.',
 'bạn sẽ ngạc nhiên khi những đứa trẻ bị ảnh hưởng bởi điều này.',
 'bạn sẽ ngạc nhiên khi trẻ con bị ảnh hưởng bởi điều này.']

In [11]:
translator("You would be surprised how kids are affected by this", beam_size=config['beam_size'], beam_return_topk=config['beam_return_topk'], max_seq_length=2)


['bạn sẽ', 'các bạn', 'bạn có thể']